### Scraping lyric data from metal-archives.com

In [27]:
import requests
import lxml.html as lh
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

In [38]:
def get_bs(url):
    try:
        headers = { 'User-Agent': 'Mozilla/5.0'
        }
        response = requests.get(url, headers=headers)
        return BeautifulSoup(response.text,'lxml')
    except ConnectionError:
        print('url')
        get_bs(url)
def bands_on_page(url):
    bs = get_bs(url)
    band_links = []
    for link in bs.find_all('a'):
        band_url = str(link).split('\"')[1]
        band_id = band_url.split('/')[-1][:-1]
        band_disco = f'https://www.metal-archives.com/band/discography/id/{band_id}/tab/all'
        band_links.append(band_disco)
    return band_links

def get_albums(url):
    bs = get_bs(url)
#print(bs)
    album_links = []
    for a in bs.find_all('a', {'class': ['albumn', 'single']}):
        link = str(a).split('\"')[3]
        album_links.append(link)
    return(album_links)

def get_lyrics(url):
    bs = get_bs(url)
    id_strings = bs.find_all('a', class_='anchor')
    ids = []
    if(len(id_strings)) == 0:
        return ([])
    for id in id_strings:
        ids.append(str(id).split('\"')[3])
    lyrics = []
    for id in ids:
        bs = get_bs(f'https://www.metal-archives.com/release/ajax-view-lyrics/id/{id}')

        lyrics.append(bs)
    return lyrics

def get_clean_lyrics(lyrics):
    if len(lyrics) == 0:
        return []
    test_lyric = str(lyrics[0])
    test_lyric = test_lyric.split('<br/>')[3:-2]
    lyrics_cleaned = []
    for i, line in enumerate(test_lyric):
        if  line != '\n':
            lyrics_cleaned.append(line[2:])
    return lyrics_cleaned


In [29]:
band_links = []
for i in range(0, 37800, 200):
    URL = f'https://www.metal-archives.com/search/ajax-advanced/searching/bands/?bandName=&genre=&country=US&yearCreationFrom=&yearCreationTo=&bandNotes=&status=&themes=&location=&bandLabelName=&sEcho=4&iColumns=3&sColumns=&iDisplayStart={i}&iDisplayLength=37800&mDataProp_0=0&mDataProp_1=1&mDataProp_2=2&_=1692040038976'
    band_links = np.concatenate((band_links, bands_on_page(URL)))

In [31]:
print(len(band_links))
albums = []
try:
    for count, band in enumerate(band_links):
        albums.append(get_albums(band))
except SSLError:
    print('ssl_error')

37776


In [39]:
lyrics = []
for i, album in enumerate(albums):
    if len(album) > 0:
        lyrics.append(get_clean_lyrics(get_lyrics(album[0])))
    if i % 100 == 0:
        print(i/len(albums))

0.0
0.0026471833968657347
0.0052943667937314694
0.007941550190597205
0.010588733587462939
0.013235916984328674
0.01588310038119441
0.018530283778060146
0.021177467174925878
0.023824650571791613
0.02647183396865735
0.029119017365523085
0.03176620076238882
0.03441338415925455
0.03706056755612029
0.03970775095298602
0.042354934349851756
0.045002117746717495
0.04764930114358323
0.050296484540448966
0.0529436679373147
0.05559085133418043
0.05823803473104617
0.0608852181279119
0.06353240152477764
0.06617958492164337
0.0688267683185091
0.07147395171537484
0.07412113511224058
0.07676831850910631
0.07941550190597205
0.08206268530283778
0.08470986869970351
0.08735705209656926
0.09000423549343499
0.09265141889030072
0.09529860228716645
0.09794578568403219
0.10059296908089793
0.10324015247776366
0.1058873358746294
0.10853451927149513
0.11118170266836086
0.11382888606522659
0.11647606946209234
0.11912325285895807
0.1217704362558238
0.12441761965268953
0.12706480304955528
0.129711986446421
0.1323591

In [59]:
for lyric in lyrics:
    if lyric == []:
        lyrics.remove([])

lyric_lined = []
for lyric in lyrics:
    lyric_lined.append('\n'.join(lyric))


lyric_array = np.array(lyric_lined)
np.savetxt('/Users/patricknaylor/Desktop/Metal/Data/lyrics_1.csv', lyric_array, delimiter=',', fmt='%s')

In [67]:
total_str = ''.join(lyric_lined)
text_file = open('/Users/patricknaylor/Desktop/Metal/Data/lyrics_1.txt', 'w')
n = text_file.write(total_str)
text_file.close()
